# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [1]:
#import my functions
import sys
sys.path.append(sys.path[0]+'/../..')


import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
from utils.timePeriod import averageForTimePeriod

In [2]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [3]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [4]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [5]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [6]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in _index.monthsOfInterest:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino34NoDetrend,nino4NoDetrend,eastIONoDetrend,dmi,nino34,nino4,eastIO,sam,lat1,lat2
year,,,,,,,,,,
1995,1.98169,2.64464,2.20035,2.53920,3.06046,2.57899,2.72196,3.50931,2.53631,2.48488
1996,2.41478,3.01493,2.15696,2.60844,2.19964,2.73435,2.82399,2.39420,2.40925,2.91225
1997,1.63983,2.31148,2.50829,2.71548,2.44572,2.43854,3.30959,2.58792,1.83214,3.19474
1998,2.92554,2.24512,2.92232,2.54869,2.40700,2.20896,2.36074,2.76448,2.06302,2.57917


Calculate it using the function

In [7]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)

calculatedAvs.round(5)

['eastIO', 'dmi', 'nino34', 'sam', 'nino4NoDetrend', 'eastIONoDetrend', 'nino34NoDetrend', 'nino4', 'lat2', 'lat1']


<xarray.Dataset>
Dimensions:          (year: 4)
Dimensions without coordinates: year
Data variables:
    eastIO           (year) float64 2.722 2.824 3.31 2.361
    dmi              (year) float64 2.539 2.608 2.715 2.549
    nino34           (year) float64 3.06 2.2 2.446 2.407
    sam              (year) float64 3.509 2.394 2.588 2.764
    nino4NoDetrend   (year) float64 2.645 3.015 2.311 2.245
    eastIONoDetrend  (year) float64 2.2 2.157 2.508 2.922
    nino34NoDetrend  (year) float64 1.982 2.415 1.64 2.926
    nino4            (year) float64 2.579 2.734 2.439 2.209
    lat2             (year) float64 2.485 2.912 3.195 2.579
    lat1             (year) float64 2.536 2.409 1.832 2.063

In [8]:
numpy.isclose(calculatedAvs.sam, expectedAv.sam)

array([ True,  True,  True,  True])

In [9]:
numpy.unique(testXr.time.dt.year)

array([1995, 1996, 1997, 1998, 1999])